#### Copied and modified from
https://github.com/aws-samples/sagemaker-distributed-training-workshop/blob/main/1_data_parallel/PyTorch%20Lightning%20on%20SageMaker.ipynb

### Run this notebook from outside the folder

In [3]:
!pip install --upgrade pip

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 27.1 MB/s eta 0:00:00:00:01
  Attempting uninstall: pip
    Found existing installation: pip 22.0.4
    Uninstalling pip-22.0.4:
      Successfully uninstalled pip-22.0.4


In [4]:
!pip install sagemaker
!pip install boto3 --upgrade

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com


In [5]:
import sagemaker
# make sure this is at least 2.102.0
sagemaker.__version__

'2.109.0'

In [7]:
from sagemaker.debugger import ProfilerConfig, FrameworkProfile

profiler_config = ProfilerConfig(
    system_monitor_interval_millis=500, framework_profile_params=FrameworkProfile(num_steps=10)
)

In [9]:
import os 
os.chdir("..")

In [ ]:
import sagemaker
from sagemaker.pytorch import PyTorch
from sagemaker.local import LocalSession

sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()
region = sagemaker_session.boto_region_name

# hard code point to the DLC images
image_uri = '763104351884.dkr.ecr.{}.amazonaws.com/pytorch-training:1.12.0-gpu-py38-cu113-ubuntu20.04-sagemaker'.format(region)

use_spot_instances = True
max_run = 1200
max_Wait = 2400 if use_spot_instances else None
checkpoint_s3_path = "sagemaker-traning-checkpoint"

estimator = PyTorch(
  entry_point="train_sagemaker.py",
  base_job_name="lightning-ddp-arxiv-hunter",
  image_uri = image_uri,
  role=role,
  source_dir="arxiv_hunter",
  # configures the SageMaker training resource, you can increase as you need
  instance_count=1,
  instance_type="ml.g4dn.12xlarge",
  py_version="py38",
  sagemaker_session=sagemaker_session,
  distribution={"pytorchddp":{"enabled": True}},
  debugger_hook_config=False,
  profiler_config=profiler_config,
  checkpoint_s3_uri = checkpoint_s3_path,
  use_spot_instances=use_spot_instances,
  max_run=max_run,
  max_wait=max_wait)

In [ ]:
estimator.fit(wait=True)

In [ ]:
# check profiler report
import sagemaker
from sagemaker.pytorch import PyTorch

rule_output_path = estimator.output_path + estimator.latest_training_job.job_name + "/rule-output"
! aws s3 cp {rule_output_path} ./ --recursive